In [1]:
import pandas as pd
import sqlalchemy
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt
from sqlalchemy.exc import SQLAlchemyError
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


from db_queries import username, password, dsn, dbhostname, service_name, dbtables, querys
from table_functions import *
from analyze_visualisation import *

In [2]:
data = {}

try:
    sqlalchemy_engine="oracle+cx_oracle://"+username+":"+password+"@"+dbhostname+"/?service_name="+service_name
    engine = sqlalchemy.create_engine(sqlalchemy_engine, arraysize=1000)
    for table, query in zip(dbtables, querys):
        data.update({table: pd.read_sql(query, engine)})
except SQLAlchemyError as e:
    print(e)

data = drop_unused_columns(data)

In [3]:
final_table = combine_final_table(data)
final_table = create_final_status(final_table)
final_table = drop_columns_not_used_in_ml(final_table)

c:\Users\DLXPQ69\Desktop\AKTUALNY PROJEKT\meb_process_data_analysis\src\table_functions.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_table['status'] = final_table['status'].replace(['3', '14'], '2')


our_final_status
1.0    662380
2.0     27445
Name: count, dtype: int64


In [4]:
final_table, categorized_columns = categorize_data(final_table)
save_df_to_csv(final_table, 'final_table_before_standarization.csv')
final_table = standarize_data(final_table)
save_df_to_csv(final_table, 'final_table_before_normalization.csv')
final_table

,czas_fazy_1,czas_fazy_2,czas_fazy_3,max_predkosc,cisnienie_tloka,cisnienie_koncowe,nachdruck_hub,anguss,oni_temp_curr_f1,oni_temp_fore_f1,...,working_mode_23_2.0,working_mode_24_2.0,working_mode_25_1.0,working_mode_25_2.0,working_mode_25_3.0,working_mode_26_1.0,working_mode_26_2.0,working_mode_27_2.0,working_mode_27_3.0,working_mode_28_2.0
0,2277.0,90.0,45.0,6.30,5.0,282.0,27.0,35.0,79.8,81.9,...,1,1,0,0,1,0,1,0,1,1
1,2220.0,76.0,16.0,5.00,4.0,281.0,0.0,41.0,71.8,70.1,...,1,1,0,0,1,0,1,1,0,1
2,2277.0,91.0,44.5,6.26,5.0,282.0,27.0,34.0,79.5,81.7,...,1,1,0,0,1,0,1,0,1,1
3,2275.0,89.0,43.5,6.37,5.0,282.0,27.0,33.0,79.8,82.0,...,1,1,0,0,1,0,1,0,1,1
4,2276.0,91.0,44.5,6.37,5.0,282.0,27.0,33.0,79.5,81.8,...,1,1,0,0,1,0,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724253,1875.0,88.0,12.0,5.00,7.0,260.0,8.0,36.0,77.6,77.0,...,1,1,0,0,0,0,1,1,0,1
724254,1830.0,90.0,12.0,5.00,3.0,239.0,8.0,37.0,75.7,75.9,...,1,1,0,0,0,0,1,1,0,1
724255,1841.0,90.0,11.0,5.00,3.0,236.0,9.0,37.0,74.3,76.2,...,1,1,0,0,0,0,1,1,0,1
724256,1808.0,89.0,10.0,5.00,3.0,258.0,9.0,37.0,75.9,74.3,...,1,1,0,0,0,0,1,1,0,1


In [6]:
final_table.iloc[:, -1]

0         1
1         1
2         1
3         1
4         1
         ..
724253    1
724254    1
724255    1
724256    1
724257    1
Name: working_mode_28_2.0, Length: 689825, dtype: int32

In [ ]:
categorized_columns.pop(-1)
ml_data = normalize_data(final_table, categorized_columns)
#for name in ['train', 'test', 'valid']:
#    save_df_to_csv(ml_data[f'x_{name}'], f'final_table_x-{name}.csv')
#    save_df_to_csv(ml_data[f'y_{name}'], f'final_table_y-{name}.csv')

In [5]:
final_table = read_csv('final_table_before_normalization.csv')

In [6]:
final_table

,czas_fazy_1,czas_fazy_2,czas_fazy_3,max_predkosc,cisnienie_tloka,cisnienie_koncowe,nachdruck_hub,anguss,oni_temp_curr_f1,oni_temp_fore_f1,...,working_mode_23_2.0,working_mode_24_2.0,working_mode_25_1.0,working_mode_25_2.0,working_mode_25_3.0,working_mode_26_1.0,working_mode_26_2.0,working_mode_27_2.0,working_mode_27_3.0,working_mode_28_2.0
0,2277.0,90.0,45.0,6.30,5.0,282.0,27.0,35.0,79.8,81.9,...,1,1,0,0,1,0,1,0,1,1
1,2220.0,76.0,16.0,5.00,4.0,281.0,0.0,41.0,71.8,70.1,...,1,1,0,0,1,0,1,1,0,1
2,2277.0,91.0,44.5,6.26,5.0,282.0,27.0,34.0,79.5,81.7,...,1,1,0,0,1,0,1,0,1,1
3,2275.0,89.0,43.5,6.37,5.0,282.0,27.0,33.0,79.8,82.0,...,1,1,0,0,1,0,1,0,1,1
4,2276.0,91.0,44.5,6.37,5.0,282.0,27.0,33.0,79.5,81.8,...,1,1,0,0,1,0,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
689402,1816.0,87.0,12.0,6.00,3.0,260.0,9.0,45.0,75.7,75.2,...,1,1,0,0,0,0,1,1,0,1
689403,1826.0,87.0,10.0,6.00,3.0,257.0,7.0,47.0,75.4,74.9,...,1,1,0,0,0,0,1,1,0,1
689404,1846.0,88.0,11.0,5.00,5.0,258.0,9.0,45.0,75.5,75.0,...,1,1,0,0,0,0,1,1,0,1
689405,1868.0,88.0,12.0,5.00,2.0,242.0,9.0,46.0,75.9,75.0,...,1,1,0,0,0,0,1,1,0,1


In [ ]:
analyze_data(final_table)

In [ ]:
ml_data = split_data(final_table)

In [ ]:
final_table_9, final_table_10 = distinct_machine(final_table)

In [6]:
clf = DecisionTreeClassifier(random_state=0)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

         1.0       0.97      0.95      0.96     66070
         2.0       0.22      0.35      0.27      2734

    accuracy                           0.92     68804
   macro avg       0.59      0.65      0.61     68804
weighted avg       0.94      0.92      0.93     68804

[[62581  3489]
 [ 1776   958]]


In [4]:
clf = RandomForestClassifier(bootstrap=True, max_depth=100, max_features=3, min_samples_leaf=4, min_samples_split=8, n_estimators=300)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

         1.0       0.97      0.98      0.98     66070
         2.0       0.40      0.25      0.31      2734

    accuracy                           0.96     68804
   macro avg       0.69      0.62      0.64     68804
weighted avg       0.95      0.96      0.95     68804

[[65045  1025]
 [ 2049   685]]


In [5]:
y_train = (y_train - 1).astype(int)
y_test = (y_test - 1).astype(int)
y_valid = (y_valid - 1).astype(int)

clf = XGBClassifier()
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.89      0.93     66070
           1       0.16      0.51      0.24      2734

    accuracy                           0.87     68804
   macro avg       0.57      0.70      0.58     68804
weighted avg       0.94      0.87      0.90     68804

[[58548  7522]
 [ 1346  1388]]
